In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, regexp_replace, lower, explode, trim, split
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
# The code can currently not run with all years and months selected
years = ["06", "07", "08", "09", "10"]
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
#months = ["01"]

In [3]:
spark = SparkSession.builder\
    .master("spark://192.168.2.97:7077") \
    .appName("claude_carlsson_hdfs")\
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
    .config("spark.shuffle.service.enabled", True)\
    .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
    .config("spark.cores.max", 5)\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/13 10:58:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def parse_json_body_to_string(json_filename):
    # Read the json file
    df = spark.read.json(json_filename)

    # Make everything lowercase
    my_df_lower = df.select(lower("body").alias("lowercase_body"))

    # Remove all charachters that exist in charachters_to_delete
    charachters_to_delete = "[\.,\[\]\(\):_\-!?\'\+=;/&{}@$#*\"\\\\%><|~¨´¤]"

    my_df_clean = my_df_lower.withColumn("lowercase_body", regexp_replace(my_df_lower.lowercase_body, charachters_to_delete, ""))

    # Parse the body to words
    my_df_words = my_df_clean.select(explode(split("lowercase_body", "\s+")).alias("word"))

    return my_df_words

In [5]:
def get_word_count_and_df_all_years():
    word_count = {}
    empty_schema = StructType([StructField("word", StringType(), True),])
    df_all_years = spark.createDataFrame([], empty_schema)

    for year in years:
        df_this_year = spark.createDataFrame([], empty_schema)

        # Append all the months to the dataframe
        for month in months:
            df_month = parse_json_body_to_string("hdfs://192.168.2.97:50000/user/ubuntu/RC_20" + year + "-" + month + ".json")
            df_this_year = df_this_year.union(df_month)

        # Count and sort the words. Save the result in word_count
        word_count_this_year = df_this_year.groupBy("word").count().orderBy("count", ascending=False)
        year_string = "20" + year
        word_count.update({year_string: word_count_this_year})

        # Sort df_this_year and update df_all_years
        df_all_years = df_all_years.union(df_this_year)
    return word_count, df_all_years

In [6]:
# Get the dictionaries
word_count, df_all_years = get_word_count_and_df_all_years()

# Find top 1000 most used words
word_count_all_years = df_all_years.groupBy("word").count().orderBy("count", ascending=False)
most_common_words = word_count_all_years.limit(500)

# Remove the most common words from the word count for each year
word_count_unique = {}
for year in years:
    year_string = "20" + year
    word_count_unique[year_string] = word_count[year_string].join(most_common_words, ["word"], "leftanti")
    # word_count_unique[year_string] = word_count_unique[year_string].sort('count', ascending=False)

In [18]:
def compare(df1, df2):
    df1.createOrReplaceTempView("df3")
    df2.createOrReplaceTempView("df4")
    
    #df = df1.select("word").filter("word")
    
    df = spark.sql("SELECT * FROM df3 WHERE " + 
                   "LEFT OUTER JOIN df4" +
                  "ON (df3.word == df4.word)" +
                  "WHERE df4.word IS NULL")
    return df
    

In [19]:
df5 = compare(word_count_unique["2007"], word_count_unique["2008"])
df5.show()

AnalysisException: cannot resolve 'df3.word' given input columns: []; line 1 pos 47;
'Project [*]
+- 'Filter isnull('df4.word)
   +- 'Join LeftOuter
      :- SubqueryAlias WHERE
      :  +- SubqueryAlias df3
      :     +- View (`df3`, [word#238,count#470L])
      :        +- Project [word#238, count#470L]
      :           +- Join LeftAnti, (word#238 = word#0)
      :              :- Sort [count#470L DESC NULLS LAST], true
      :              :  +- Aggregate [word#238], [word#238, count(1) AS count#470L]
      :              :     +- Union false, false
      :              :        :- LogicalRDD [word#238], false
      :              :        :- Project [word#256]
      :              :        :  +- Generate explode(split(lowercase_body#253, \s+, -1)), false, [word#256]
      :              :        :     +- Project [regexp_replace(lowercase_body#251, [\.,\[\]\(\):_\-!?'\+=;/&{}@$#*"\\%><|~¨´¤], , 1) AS lowercase_body#253]
      :              :        :        +- Project [lower(body

In [ ]:
spark.stop()